In [0]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
style.use('ggplot')


In [0]:
# Function to validate input by data type
def check_input(f_type,message):
  flag = True
  while flag:
    value = input(message)
    try:
      value = f_type(value)
      flag = False
    except ValueError:
      pass
  return value

# Read the data:
# npoints with nrepl replicates eacht.
# Repeat a line by answerting "No" to "Input OK?"
npoints = check_input(int,"Number of points? ")
nrepl = check_input(int,"Number of replicates? ")
s = np.zeros(npoints)
v = np.zeros((npoints, nrepl))
i = 0
while i < npoints:
  s[i] = check_input(float,"Substrate concentration? ")
  for j in range(nrepl):
    v[i,j] = check_input(float,"Velocity(Replicate {})? ".format(j+1))
  bq = input("Input OK? (y/n)")
  if bq.upper()[0] == "N":
    i = i - 1
  i = i + 1
      
# print the data
np.hstack((s.reshape(-1,1),v))
  

In [0]:
#@title Example data (uncomment)
#
# Example data
# x = np.array([[ 1.   , -1.003,  6.868,  4.254],
#              [ 6.   , 24.747, 27.017, 25.237],
#              [11.   , 36.619, 38.776, 42.157],
#              [16.   , 39.641, 43.188, 48.65 ],
#              [21.   , 47.674, 50.211, 53.12 ],
#              [26.   , 53.475, 56.319, 48.891],
#              [31.   , 57.83 , 55.559, 57.094],
#              [36.   , 58.634, 57.599, 55.835],
#              [41.   , 58.942, 61.771, 65.595],
#              [46.   , 63.796, 62.747, 65.087],
#              [51.   , 61.448, 59.106, 68.988],
#              [56.   , 65.13 , 69.53 , 62.655]])
#
# s = x[:,0]
# v = x[:,1:]
# np.hstack((s.reshape(-1,1),v))
# nrepl = 3
# npoints = 12

In [0]:
# create a pandas data frame with the data
# also works:
# enz_data.iloc[:,1:4].mean(axis=1)
# enz_data.iloc[:,1:4].std(axis=1)
#
columns = ['[A]']
columns = columns + ['v'+str(i+1) for i in range(nrepl)]
enz_data = pd.DataFrame(np.hstack([s.reshape(-1,1),v]),columns=columns)
enz_data['Mean v']=enz_data[['v1','v2','v3']].mean(axis=1)
enz_data['Std v']=enz_data[['v1','v2','v3']].std(axis=1)
#enz_data['fit_error']=enz_data['Mean v']-MM(Km,Vmax,enz_data['[A]'])
enz_data

#### Create the Direct Linear Plot and Compute the Median Estimates of Km and Vmax

In [0]:
style.use('default')
vm = v.mean(axis=1)
x = np.arange(-s.max(),s.max(),0.01)
Vmax_intersect = []
Km_intersect = []
mmlines = np.vstack((vm/s,vm)).T
n_int = len(s)*(len(s)-1)//2
Km_int = np.zeros(n_int)
Vmax_int = np.zeros(n_int)

for m, b in mmlines:
  plt.plot(x,m*x+b)

print("Computing {:d} intersections".format(n_int))
for i in range(mmlines.shape[0]):
  for j in range(i):
    m1, b1 = mmlines[i]
    m2, b2 = mmlines[j]
    x = (b2-b1)/(m1-m2)
    y = (b2*m1-b1*m2)/(m1-m2)
    Km_intersect.append(x)
    Vmax_intersect.append(y)
    #print(x,y)
    #pass
    #print(i,j,mmlines[i],mmlines[j])

ax = plt.gca()
ax.axis([-1.1*s.max(),1.1*s.max(),-2*vm.max(),2*vm.max()])
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
#ax.set_xlabel("Km")
#ax.set_ylabel("Vmax")
#ax.set_ylabel?
print("Median Estimate of Km: {:5.2f}".format(np.median(Km_intersect))) 
print("Median Estimate of Vmax: {:5.2f}".format(np.median(Vmax_intersect)))  